In [1]:
cd py-hanspell-master

c:\Users\qudwn\OneDrive\바탕 화면\2023summer_vac_project\aihub 에세이 데이터\py-hanspell-master


In [2]:
!python setup.py install      # py-hanspell-master 파일을 실행하려는 주피터 노트북이랑 같은 폴더 안에 놓고 코드 돌리면 됨

running install
running bdist_egg
running egg_info
writing py_hanspell.egg-info\PKG-INFO
writing dependency_links to py_hanspell.egg-info\dependency_links.txt
writing requirements to py_hanspell.egg-info\requires.txt
writing top-level names to py_hanspell.egg-info\top_level.txt
reading manifest file 'py_hanspell.egg-info\SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'py_hanspell.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
creating build\bdist.win-amd64\egg
creating build\bdist.win-amd64\egg\hanspell
copying build\lib\hanspell\constants.py -> build\bdist.win-amd64\egg\hanspell
copying build\lib\hanspell\response.py -> build\bdist.win-amd64\egg\hanspell
copying build\lib\hanspell\spell_checker.py -> build\bdist.win-amd64\egg\hanspell
copying build\lib\hanspell\__init__.py -> build\bdist.win-amd64\egg\hanspell
byte-compiling build\bdist.win-amd64\egg\hanspell\constants.py to constants.cpython-39.pyc
byt

c:\Users\qudwn\anaconda3\envs\2023_summer_vac_first_project\lib\site-packages\setuptools\dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separated 'description-file' will not be supported in future
        versions. Please use the underscore name 'description_file' instead.

        By 2023-Sep-26, you need to update your project and remove deprecated calls
        or your builds will no longer be supported.

        See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
        ********************************************************************************

!!
  opt = self.warn_dash_deprecation(opt, section)
c:\Users\qudwn\anaconda3\envs\2023_summer_vac_first_project\lib\site-packages\setuptools\_distutils\cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ***********************

In [3]:
from hanspell import spell_checker
result = spell_checker.check('안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.')
result.as_dict()  # dict로 출력

{'result': True,
 'original': '안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.',
 'checked': '안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.',
 'errors': 4,
 'words': OrderedDict([('안녕하세요.', 2),
              ('저는', 0),
              ('한국인입니다.', 2),
              ('이', 2),
              ('문장은', 2),
              ('한글로', 0),
              ('작성됐습니다.', 1)]),
 'time': 0.08046579360961914}

In [4]:
import pandas as pd
import numpy as np
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud
import re
from tqdm import tqdm

In [13]:
df = pd.read_csv('C:/Users/qudwn/OneDrive/바탕 화면/2023summer_vac_project/aihub 에세이 데이터/에세이데이터2.csv')
df.head()

,paragraph_txt,essay_type,student_grade,paragraph_scoreT_avg,essay_main_subject,paragraph_len,org_coherence,con_clearance,con_novelty,con_description,grade
0,익숙함에 속아 소중함을 잊지말자라는 명언은 매우 유명한 명언이다. 난 이 명언의 내...,글짓기,고등_1학년,2.555555,소중함을 잃었던 경험,1300,2,4,3,1,C
1,소중함이란 가치를 따질 수 없을 만큼 매우 귀중한 것을 말합니다. 저에게도 이러한 ...,글짓기,고등_1학년,2.444444,소중함을 잃었던 경험,1149,2,4,3,1,C
2,"""익숙함에 속아 소중함을 잊지 말자"" 모든 사람들이 아는 말이다. 그만큼 모두가 그...",글짓기,고등_1학년,2.611111,소중함을 잃었던 경험,1150,2,4,3,1,C
3,나는 가까운 사람에 대한 소중함에 대하여 알지 못했다. 그 일이 있기 전까지 말이다...,글짓기,고등_1학년,2.777778,소중함을 잃었던 경험,1641,2,4,3,1,B
4,나는 태어났을 때부터 할머니와 함께 살아왔는데 할머니는 하루도 빠짐없이 나에게 아침...,글짓기,고등_1학년,3.000000,소중함을 잃었던 경험,1293,2,4,3,1,A


In [14]:
df['paragraph_txt'] = df['paragraph_txt'].apply(lambda x: str(x).replace('\n', ' '))
df['paragraph_txt'] = df['paragraph_txt'].apply(lambda x: str(x).replace('\\', '\''))

##### 500자 이상의 문장 입력받아도 맞춤법 분석 할 수 있게 spell_check_over500 함수 만들었음 - modified_sencence(맞춤법 수정된 문장),modified_words(수정이 이루어진 단어들), errors(오류 단어 개수) 정보를 딕셔너리 형태로 반환, 이 함수만 써도 될듯

In [18]:
import re

def split_sentences(text):
    sentences = re.split(r'(?<=[.?!])\s', text)
    result = []
    current_sentence = ''
    
    for sentence in sentences:
        if len(current_sentence) + len(sentence) + 1 <= 500:  # +1 for space after sentence
            current_sentence += sentence + ' '
        else:
            if current_sentence:
                result.append(current_sentence.strip())
            current_sentence = sentence + ' '
    
    if current_sentence:
        result.append(current_sentence.strip())
    
    return result


def spell_check_over500(text):
    total_modified_sentence = ''
    total_errors = 0
    total_modified_words = []

    
    sentence_list = split_sentences(text)
    for sentence in sentence_list:
        modified_dic = spell_checker.check(sentence)
        total_modified_sentence += modified_dic.as_dict()['checked']
        total_errors += modified_dic.as_dict()['errors']
        modified_words = [key for key, value in modified_dic.as_dict()['words'].items() if value != 0]
        total_modified_words += modified_words
    
    return {'modified_sencence':  total_modified_sentence, 'modified_words': total_modified_words, 'errors': total_errors}


In [19]:
spell_check_over500(df['paragraph_txt'][185])

{'modified_sencence': '마스크를 쓴 모습들이 이제는 익숙해졌지만 나는 아직도 어색하다. 방역수칙을 지키기 위한 행동들 하나하나가 나에게는 불편하고 어색하게 다가왔다. 온라인 수업에 그새 익숙해졌는지 금방 피곤해지기도 했다. 온라인과 오프라인을 섞어서 하다 보니 혼란스러웠다. 시험은 다가오는데 공부를 하고 싶은 마음은 좀처럼 들지 않고 공부를 해도 잘되지 않았다. 그러다 보니 결과는 항상 받던 점수보다 낮게 나왔고 우울해졌던 것 같다. 코로나로 인한 변화에 적응을 한 줄 알았지만 적응을 한 척했던 것 같다. 지금의 나도 완전히 적응했다고 보기는 힘들다. 하지만 지금은 예전보다는 이 상황을 잘 극복해나가는 것 같다. 시험이 끝나고 여름방학 왔다. 방학 때 나는 상황을 잘 극복해 보자는 마음으로 공부도 조금씩 하고 약간의 취미생활도 만들었다. 오히려 코로나 전의 방학보다 알차게 보낸 기분이었다. 하지만 한순간에 나태해진 나를 변화시키는 것을 어려운 일이었다.그래서 개학을 하고 나서도, 겨울 한 오고도 나는 계획적으로 열심히 살고자 노력했다. 꾸준히 인강도 보고 공부도 하고 운동도 하며 시간을 보냈고 양모로 인형 만들기 등 여러 취미 생활도 했다. 또한 이번에는 진로를 위해서 여러 영상들도 찾아보고 책들도 많이 읽으면서 유익한 시간을 보냈다. 코로나 사태로 익숙한 것들을 많이 잃었다. 이런 상황을 극복하기 위해서 많은 노력을 했고 앞으로도 할 예정이다.',
 'modified_words': ['익숙해졌지만',
  '잘되지',
  '한',
  '척했던',
  '극복해',
  '보자는',
  '나서도,',
  '겨울',
  '한'],
 'errors': 6}

In [15]:
df_a = df[df['grade'] == 'A']
df_a.reset_index(drop=True)
df_a.head()

,paragraph_txt,essay_type,student_grade,paragraph_scoreT_avg,essay_main_subject,paragraph_len,org_coherence,con_clearance,con_novelty,con_description,grade
0,나는 태어났을 때부터 할머니와 함께 살아왔는데 할머니는 하루도 빠짐없이 나에게 아침...,글짓기,고등_1학년,3.0,소중함을 잃었던 경험,1293,2,4,3,1,A
1,저는 너무하게도 익숙함에 속아서 소중함을 잃었던 사례들이 그렇게 많지는 않지만 딱 ...,글짓기,고등_1학년,3.0,소중함을 잃었던 경험,1459,2,4,3,1,A
2,요즘엔 소중하다라는 표현을 잘 쓰지 않는다. '소중하다'보다는 '필요하다'가 더 많...,글짓기,고등_1학년,3.0,소중함을 잃었던 경험,2272,2,4,3,1,A
3,"일상이란 단어의 정의를 알고있는가? 일본의 상사라는 뜻도 있고, 불교에서는 별이 없...",글짓기,고등_1학년,3.0,소중함을 잃었던 경험,401,2,4,3,1,A
4,중학교 3학년 시절 단짝 친구를 잃은 경험이 있었다. 고등학교 진학을 앞둔 상황에서...,글짓기,고등_1학년,3.0,소중함을 잃었던 경험,134,2,4,3,1,A
...,...,...,...,...,...,...,...,...,...,...,...
7947,요즘 뭔가 날씨가 굉장히 이상하다고 느낍니다. 분명 엄청 추워야하는 겨울인데 작년 ...,찬성반대,초등_6학년,3.0,에너지 절약에 대한 본인의 생각,619,2,4,2,2,A
7948,저는 에너지는 우리가 살아가면서 꼭 필요한거라고 생각하고있습니다. 그런 에너지가 지...,찬성반대,초등_6학년,3.0,에너지 절약에 대한 본인의 생각,603,2,4,2,2,A
7949,우리는 일상생활을 하는데 에너지를 많이 씁니다. 우리가 가장 많이 사용하는 핸드폰도...,찬성반대,초등_6학년,3.0,에너지 절약에 대한 본인의 생각,632,2,4,2,2,A
7950,제가 생각하는 에너지 절약에 대한 제 생각은 편리한 생활을 줄이고 에너지를 절약하여...,찬성반대,초등_6학년,3.0,에너지 절약에 대한 본인의 생각,686,2,4,2,2,A


##### 아래는 토큰화 전 데이터에 대하여 맞춤법 수정된 문장을 저장하는 코드(토큰화 후 데이터분석 결과가 잘 안나올 경우 시도가능.. 시간이 남을경우...)


In [36]:

Dataset = {'modified_sencence': [], 'modified_words': [], 'errors': []}
for sentence in tqdm(df_a['paragraph_txt'], total=len(df_a['paragraph_txt']), ## 전체 진행수
              desc='Description', ## 진행률 앞쪽 출력 문장
              ncols=70, ## 진행률 출력 폭 조절
              ascii=' =', ## 바 모양, 첫 번째 문자는 공백이어야 작동
              leave=True): ## True 반복문 완료시 진행률 출력 남김. False 남기지 않음
    
    result = spell_check_over500(sentence)
    Dataset['modified_sencence'].append(result['modified_sencence'])
    Dataset['modified_words'].append((result['modified_words']))
    Dataset['errors'].append((result['errors']))
    

Description:   2%|               | 185/7952 [02:08<1:30:03,  1.44it/s]


ParseError: not well-formed (invalid token): line 1, column 656 (<string>)

In [ ]:
Dataset['modified_words'][3]

In [ ]:
import json

with open("./essay2_tokenized.json", "w") as outfile:
    json.dump(Dataset, outfile)

In [ ]:
import json
 
with open("./essay2_tokenized.json") as json_file:
    data = json.load(json_file)